# ☁️ Lab activity 1: Unsupervised clustering

**Author**: [Andres J. Sanchez-Fernandez](https://www.unir.net/profesores/andres-jesus-sanchez-fernandez/)

**References**:

> KMEANS. (n.d.). Scikit-learn. https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

> Demo of DBSCAN clustering algorithm. (n.d.). *Scikit-learn*. https://scikit-learn.org/stable/auto_examples/cluster/plot_dbscan.html


## Objectives of this session

By the end of this lab, students will be able to:

1. Understand the concept of unsupervised learning and how it differs from supervised approaches.

1. Apply clustering algorithms such as K-Means or DBSCAN to real datasets.

1. Preprocess and explore datasets to identify appropriate features for clustering.

1. Interpret clustering results using visualizations and relevant metrics (e.g., silhouette score).

1. Justify the choice of a clustering algorithm based on the characteristics of the data.

1. Evaluate the performance and limitations of different clustering methods.

## Importing the required libraries

In [ ]:
# When executing locally.
!pip3 install numpy pandas matplotlib scikit-learn

In [ ]:
# Importing the required libraries.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import random
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import (
    silhouette_score,
    calinski_harabasz_score,
    davies_bouldin_score,
    adjusted_rand_score
)
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from typing import Dict, Tuple, List

# Reproducibility.
seed = 42
np.random.seed(42)
random.seed(42)

To make your exported script work even in standard Python environments, add a fallback for display():

In [ ]:
# Fallback if IPython is not available.
try:
    from IPython.display import display
except ImportError:
    def display(obj):
        print(obj)

## Preprocessing steps

### Target _dataset_



> Tsanas, A. & Xifara, A. (2012). Energy Efficiency [Dataset]. UCI Machine Learning Repository. https://doi.org/10.24432/C51307.





> We perform energy analysis using 12 different building shapes simulated in Ecotect. The buildings differ with respect to the glazing area, the glazing area distribution, and the orientation, amongst other parameters. We simulate various settings as functions of the afore-mentioned characteristics to obtain 768 building shapes. The dataset comprises 768 samples and 8 features, aiming to predict two real valued responses. It can also be used as a multi-class classification problem if the response is rounded to the nearest integer.



In [ ]:
# URL to download the dataset.
dataset_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx'

### Loading the dataset into a Pandas DataFrame

In [ ]:
# Read the dataset into a pandas dataframe.
df = pd.read_excel(dataset_url)

# Display the whole dataframe.
display(df)

### Data cleaning and inspection

In [ ]:
# Define array of column names.
feature_names = [
    'Relative_Compactness',
    'Surface_Area',
    'Wall_Area',
    'Roof_Area',
    'Overall_Height',
    'Orientation',
    'Glazing_Area',
    'Glazing_Area_Distribution',
    'Heating_Load',
    'Cooling_Load'
]

# Replace column names.
df.columns = feature_names

In [ ]:
# Check data types and missing values.
print('Dataset info:')
display(df.info())

# Parse all columns automatically based on their values (not necessary here).
df = df.convert_dtypes()

In [ ]:
df['Orientation'].value_counts()

In [ ]:
# Inspect numerical data.
print('Summary statistics:')
display(df.describe())

In [ ]:
# Count missing values per column.
print('Missing values per column:')
display(df.isna().sum())

In [ ]:
# Remove duplicated rows.
print(f'Number of duplicated rows: {df.duplicated().sum()}')
df = df.drop_duplicates()

### Select features

#### Categorical features

In [ ]:
# Drop categorical features for clustering.
df_clust = df.drop(columns=['Orientation', 'Glazing_Area_Distribution'])

#### Analyze correlations

In [ ]:
# Useful function to find pairs of variables with absolute correlation above the threshold.
def find_highly_correlated_pairs(
    corr_matrix: pd.DataFrame,
    threshold: float = 0.85
) -> List[Tuple[str, str, float]]:
    """
    Identifies all pairs of variables in the correlation matrix that have
    an absolute Pearson correlation greater than the specified threshold.

    Args:
        corr_matrix (pd.DataFrame): A square correlation matrix (e.g., from df.corr()).
        threshold (float): The minimum absolute correlation to consider.

    Returns:
        List[Tuple[str, str, float]]: Pairs of column names and their correlation value.
    """
    high_corr_pairs = []

    # Iterate over the columns.
    for i in range(len(corr_matrix.columns)):

        # Avoid duplicate pairs and self-correlation.
        for j in range(i):

            # Get correlation value.
            corr_value = corr_matrix.iloc[i, j]

            # Check if high correlation.
            if abs(corr_value) >= threshold:
                col_i = corr_matrix.columns[i]
                col_j = corr_matrix.columns[j]
                high_corr_pairs.append((col_i, col_j, corr_value))

    return high_corr_pairs

In [ ]:
# Find columns with at least one absolute correlation above threshold.
threshold = 0.9

In [ ]:
# Compute correlation matrix.
df_pearson_corr = df_clust.corr(numeric_only=True, method='pearson')
upper_triangle_mask = np.triu(np.ones_like(df_pearson_corr, dtype=bool))

# Plot masked correlation heatmap.
plt.figure(figsize=(10, 5))
sns.heatmap(
    df_pearson_corr,
    mask=upper_triangle_mask,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    vmin=-1, vmax=1,
    linewidths=0.5,
    cbar_kws={'shrink': 0.8},
)
plt.title('Correlation matrix (Pearson) – Bottom triangle')
plt.tight_layout()
plt.show()

# Compute high-correlation pairs from the Pearson correlation matrix.
high_corr_pairs = find_highly_correlated_pairs(df_pearson_corr, threshold)
if high_corr_pairs:
    print('Highly correlated variable pairs (Pearson):')
    for var1, var2, corr in high_corr_pairs:
        print(f' - {var1} and {var2}: correlation = {corr:.2f}')
else:
    print(f'No variable pairs with correlation above {threshold} using Pearson.')

In [ ]:
# Drop correlated features (could be dimensionality reduction as well).
# 'Relative_Compactness' is derived from surface area and volume.
# 'Heating_Load' is redundant with 'Cooling_Load'.
# 'Roof_Area' is redundant with 'Overall_Height'.
df_clust = df_clust.drop(columns=['Heating_Load', 'Relative_Compactness', 'Roof_Area'])

In [ ]:
# Create a mask to hide the lower triangle.
df_pearson_corr = df_clust.corr(numeric_only=True, method='pearson')
upper_triangle_mask = np.triu(np.ones_like(df_pearson_corr, dtype=bool))

# Plot masked correlation heatmap.
plt.figure(figsize=(10, 5))
sns.heatmap(
    df_pearson_corr,
    mask=upper_triangle_mask,
    annot=True,
    fmt='.2f',
    cmap='coolwarm',
    vmin=-1, vmax=1,
    linewidths=0.5,
    cbar_kws={'shrink': 0.8},
)
plt.title('Correlation matrix (Pearson) – Bottom triangle')
plt.tight_layout()
plt.show()

In [ ]:
# Cleaned dataset.
display(df_clust)

### Standardize features

In [ ]:
# Define the pipeline.
pipeline = Pipeline([
    ('scaler', StandardScaler())  # Step 1 in the pipeline.
])

# Fit and transform data.
data_scaled = pipeline.fit_transform(df_clust)
print(data_scaled)

## K-Means

### Determine the optimal $k$ for K-Means

#### Elbow method

The Elbow Method evaluates the optimal number of clusters by measuring the inertia (within-cluster sum of squares). The goal is to find the "elbow point," where the inertia decreases significantly but levels off, indicating diminishing returns for additional clusters.

Inertia measures the **within-cluster sum of squares (WCSS)** and evaluates how well the clustering minimizes the distance between data points and their assigned cluster centroid. It is a measure of the compactness of clusters.

For a given clustering solution, inertia is calculated as:

$$
\text{Inertia} = \sum_{k=1}^{K} \sum_{i \in C_k} \| x_i - \mu_k \|^2
$$

where:
- $ K $: Number of clusters.
- $ C_k $: The set of points in cluster $k$.
- $ x_i $: A data point.
- $ \mu_k $: The centroid of cluster $k$.
- $ \| x_i - \mu_k \|^2 $: Squared Euclidean distance between a point and its cluster centroid.

In [ ]:
# Range of clusters to evaluate.
k_range = range(2, 10)

Start the Elbow and Silhouette analysis at $k=2$, not $k=1$, since $k=1$ gives no useful clustering insight: inertia is always highest and the silhouette score is undefined. Only $k\geq2$ reveals meaningful cluster structure.

In [ ]:
# Compute K-Means and inertia for several ks.
inertias = []

# Iterate over the clusters.
for k in k_range:

    # Compute k-means for a given k.
    kmeans = KMeans(n_clusters=k, random_state=seed)
    kmeans.fit(data_scaled)

    # Save inertia.
    inertias.append(kmeans.inertia_)

# Visualization of the elbow method.
plt.figure(figsize=(8, 5))
plt.plot(k_range, inertias, marker='o', linestyle='--')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia (within-cluster sum of squares)')
plt.title('Elbow method for k-means')
plt.show()

* The inertia **drops rapidly from $k=2$ to $k=6$**, after which the decrease slows down.

* This suggests an **elbow at $k=6$**, which is often interpreted as the optimal point because adding more clusters beyond that brings diminishing returns in variance explained

In [ ]:
!pip3 install kneed

In [ ]:
from kneed import KneeLocator

# Using KneeLocator to automatically find the elbow point.
kneedle = KneeLocator(k_range, inertias, curve='convex', direction='decreasing')
print(f'Optimal number of clusters: {kneedle.elbow}')

#### Silhouette score

The Silhouette Score measures the quality of clustering by comparing how similar each point is to its own cluster (cohesion) versus other clusters (separation). Scores range from -1 to 1, where higher scores (close to 1) indicate better-defined clusters.


For each data point $i$:
1. **Cohesion $a(i)$:**
   - Compute the average distance between $i$ and all other points in its cluster.
2. **Separation $b(i)$:**
   - Compute the average distance between $i$ and all points in the nearest neighboring cluster (the next closest cluster to $i$).

The silhouette score for point $i$ is:

$$
s(i) = \frac{b(i) - a(i)}{\max(a(i), b(i))}
$$

In [ ]:
# Compute K-Means and silhouette_scores for several ks.
silhouette_scores = []

# Iterate over the clusters.
for k in k_range:

    # Compute k-means for a given k.
    kmeans = KMeans(n_clusters=k, random_state=seed)
    kmeans.fit(data_scaled)

    # Save score.
    silhouette_scores.append(silhouette_score(data_scaled, kmeans.labels_))

# Visualization of silhouette score.
plt.figure(figsize=(8, 5))
plt.plot(k_range, silhouette_scores, marker='o', linestyle='--')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')
plt.title('Silhouette analysis for k-means')
plt.show()

* The silhouette score is **highest at $k=2$**, meaning those clusters are the most compact and well-separated.

* However, **$k=2$ may oversimplify the structure**, especially if you expect more natural groupings.

### Selection of optimal $k$

Although the elbow point is in 6 clusters, we will select $k=2$ to compare these clusters to those found by DBSCAN using ARI, which resulted in $k=2$ having the highest Silhouette score. **This is a conservative but insightful initial scenario.**

In [ ]:
# Set the optimal number of clusters.
k_optimal = 2

### Inspecting found clusters

In [ ]:
# Applying k-means.
kmeans = KMeans(n_clusters=k_optimal, random_state=seed)

# Compute and predict clusters for each sample in DataFrame.
df['Cluster_k-means'] = kmeans.fit_predict(data_scaled)

In [ ]:
# Count number of samples per cluster.
cluster_counts = df['Cluster_k-means'].value_counts().sort_index()

# Plot.
cluster_counts.plot(kind='bar', figsize=(8, 5))
plt.xlabel('Cluster label')
plt.ylabel('Number of Samples')
plt.title('K-means cluster sizes')
plt.show()

In [ ]:
# Non-scaled centroids.
centroids_not_scaled = df.groupby('Cluster_k-means').mean().round(2)

In [ ]:
# Plot data points and centroids.
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df['Heating_Load'], y=df['Cooling_Load'], hue=df['Cluster_k-means'], palette='Set1')
plt.scatter(centroids_not_scaled['Heating_Load'], centroids_not_scaled['Cooling_Load'], color='black', marker='*', s=200, label='Non-scaled centroids')
plt.title('Clusters with k-means')
plt.xlabel('Heating load')
plt.ylabel('Cooling load')
plt.show()

### Interpretation of centroids

In [ ]:
# Resulting non-scaled centroids.
print('Cluster centroids not scaled:')
display(centroids_not_scaled)

In [ ]:
# See how clusters are distributed across feature pairs.
sns.pairplot(df, hue='Cluster_k-means', diag_kind='hist', corner=True)
plt.show()

## DBSCAN

### Tune hyperparameters

| Situation                  | Suggested Action     |
|---------------------------|----------------------|
| Too many small clusters   | Increase `eps` slightly |
| Too many noise points (-1)| Decrease `min_samples` |

#### minPts & epsilon with elbow method

In [ ]:
# Set minimum number of points to form a cluster.
minPts = data_scaled.shape[1] + 1
print(f'Minimum number of points to form a cluster: {minPts}')

Set `k = minPts`. Then use the **k-distance plot** to visually identify the ideal **ε** value (look for the “elbow”).

In [ ]:
# Use NearestNeighbors to compute distances to k-th nearest neighbors.
k = minPts
neighbors = NearestNeighbors(n_neighbors=k)
neighbors_fit = neighbors.fit(data_scaled)
distances, indices = neighbors_fit.kneighbors(data_scaled)

# Sort the distances to the k-th neighbor (use for elbow/knee method).
k_distances = np.sort(distances[:, -1])

# Plot the k-distance graph.
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=k_distances,
    mode='lines+markers',
    line=dict(color='royalblue'),
    marker=dict(size=5),
    name=f'{k}-th Nearest Neighbor Distance'
))

fig.update_layout(
    title=f'{k}-Distance plot (for ε selection)',
    xaxis_title=f'Points sorted by distance to {k}th nearest neighbor',
    yaxis_title='Distance',
    template='plotly_white',
    hovermode='x unified',
    height=400,
    width=800
)

fig.show()

We visually inspect the eps range on the graph to set a good grid search criteria.

In [ ]:
# Using KneeLocator to automatically find the elbow point.
kneedle = KneeLocator(range(len(k_distances)), k_distances, curve='convex', direction='increasing')
print(f'Elbow of eps: {k_distances[kneedle.elbow]}')

#### Grid Search

Grid search optimizes Silhouette Score, finding clusters that are internally tight and externally separated.

In [ ]:
# Define parameter grids.
eps_values = np.linspace(0.7, 2.0, 20)
min_samples_values = range(data_scaled.shape[1]+1, data_scaled.shape[1]*2)

# Variables to store best results.
best_score = -1
best_eps = None
best_min_samples = None
best_n_clusters = None
best_model = None

# Grid Search.
for eps in eps_values:
    for min_samples in min_samples_values:

        # Fit model.
        model = DBSCAN(eps=eps, min_samples=min_samples)
        labels = model.fit_predict(data_scaled)

        # Ignore noise.
        mask = labels != -1
        n_clusters = len(set(labels[mask]))

        # Silhouette score needs at least 2 clusters.
        if n_clusters <= 1:
            continue

        # Compute Silhouette.
        try:
            score = silhouette_score(data_scaled[mask], labels[mask])
        except Exception as e:
            print(f'Error at eps={eps:.2f}, min_samples={min_samples}: {e}')
            continue
        print(f'eps={eps:.2f}, min_samples={min_samples}, n_clusters={n_clusters}, Silhouette={score:.4f}')

        # Save best result so far.
        if score > best_score:
            best_score = score
            best_eps = eps
            best_min_samples = min_samples
            best_n_clusters = n_clusters
            best_model = model

In [ ]:
# Final best parameters
print('Best combination found:')
print(f'eps = {best_eps:.4f}')
print(f'min_samples = {best_min_samples}')
print(f'Number of clusters = {best_n_clusters}')
print(f'Best Silhouette score = {best_score:.4f}')

### Inspecting found clusters

In [ ]:
# Save labels and compute Silhouette without noise.
df['Cluster_DBSCAN'] = best_model.labels_
mask = best_model.labels_ != -1
silhouette = silhouette_score(data_scaled[mask], df['Cluster_DBSCAN'][mask])
print(f'Silhouette score: {silhouette:.4f}')

In [ ]:
# Count number of samples per cluster.
cluster_counts = df['Cluster_DBSCAN'].value_counts().sort_index()

# Plot.
cluster_counts.plot(kind='bar', figsize=(8, 5))
plt.xlabel('Cluster label')
plt.ylabel('Number of Samples')
plt.title('DBSCAN cluster sizes')
plt.show()

In [ ]:
# Plot data points.
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df['Heating_Load'], y=df['Cooling_Load'], hue=df['Cluster_DBSCAN'], palette='Set1')
plt.title('Clusters with DBSCAN')
plt.xlabel('Heating load')
plt.ylabel('Cooling load')
plt.show()

### Interpretation of average groups

In [ ]:
# Convert to DataFrame.
mean_groups = pd.DataFrame(df, columns=df.drop(columns=['Cluster_k-means']).columns).groupby('Cluster_DBSCAN').mean().round(2).reset_index()
print('Cluster means:')
display(mean_groups)

In [ ]:
# See how clusters are distributed across feature pairs.
sns.pairplot(df, hue='Cluster_DBSCAN', diag_kind='hist', corner=True)
plt.show()

## Conclusion

In [ ]:
# Show DataFrame with labels.
clustering_labels = ['Cluster_k-means', 'Cluster_DBSCAN']
display(df[clustering_labels].head())

**Silhouette score**

- Measures how similar a sample is to its own cluster compared to others.
- **Ranges from -1 to 1**: higher values mean **better and well-separated** clusters.

**Calinski-Harabasz index (CH score)**

- Measures the ratio of between-cluster dispersion to within-cluster dispersion.
- **Higher values** indicate **better-defined** and **more separated** clusters.

**Davies-Bouldin index (DB score)**

- Measures the average similarity between each cluster and its most similar one.
- **Lower values** indicate **better clustering** with more distinct groups.

In [ ]:
# Initialize results list.
results = []

# Iterate over clustering label columns.
for label_col in clustering_labels:

    # Get clusters.
    labels = df[label_col].values

    # Remove noise.
    mask = labels != -1

    # Compute metrics.
    silhouette = silhouette_score(data_scaled[mask], labels[mask])
    calinski = calinski_harabasz_score(data_scaled[mask], labels[mask])
    davies = davies_bouldin_score(data_scaled[mask], labels[mask])

    # Append results.
    results.append({
        'Clustering': label_col,
        'Silhouette': silhouette,
        'Calinski-Harabasz': calinski,
        'Davies-Bouldin': davies
    })

# Create summary DataFrame.
metrics_df = pd.DataFrame(results)
display(metrics_df)

In [ ]:
# Create a copy to avoid modifying the original.
ranking_df = metrics_df.copy()

# For each metric, compute the ranking.
# For Silhouette and Calinski, higher is better, so descending.
# For Davies, lower is better, so ascending.
ranking_df['Silhouette_Rank'] = ranking_df['Silhouette'].rank(ascending=False).astype(int)
ranking_df['Calinski_Rank'] = ranking_df['Calinski-Harabasz'].rank(ascending=False).astype(int)
ranking_df['Davies_Rank'] = ranking_df['Davies-Bouldin'].rank(ascending=True).astype(int)

# Keep only name and rankings.
final_ranking = ranking_df[['Clustering', 'Silhouette_Rank', 'Calinski_Rank', 'Davies_Rank']]
display(final_ranking)

In [ ]:
# Compute Adjusted Rand Index.
print(f"ARI index: {adjusted_rand_score(df['Cluster_k-means'], df['Cluster_DBSCAN'])}")